<a href="https://colab.research.google.com/github/aleks-haksly/Simulative/blob/main/Other/Analysis/%D0%90%D0%BB%D1%8C%D1%84%D0%B0%20%D0%91%D0%B0%D0%BD%D0%BA%20test/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install pandasql -q

  Preparing metadata (setup.py) ... done


In [23]:
import pandas as pd
from pandasql import sqldf

%load_ext sql
%config SqlMagic.style = '_DEPRECATED_DEFAULT'

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


## SQL1
Отобрать клиентов по г. Москва с суммарными остатками по клиенту от 20 000 на
последнюю дату.

create table #Клиенты (client_id varchar(1), FIO varchar (255) , Region varchar(50), account_num int)


insert into #Клиенты values
('A','Иванов','Москва',111),
('A','Иванов','Москва',222),
('B','Петров','Иваново',333),
('C','Сидоров','Москва',444)

create table #Cчета ([Date] date, Summa_USD money, Account_num int)


insert into #Cчета values
('2012-01-01',15000,111),
('2012-02-01',10000,111),
('2012-02-01',5000,222),
('2012-03-01',30000,333),
('2012-04-01',20000,444)

### Создадим пустую БД sqlite и добавим туда нужные данные

In [11]:
%sql sqlite:////content//tempt.db

In [17]:
%%sql
CREATE TABLE Clients (client_id varchar(1), FIO varchar (255) , Region varchar(50), account_num int);
INSERT INTO Clients (client_id, FIO, Region, account_num) VALUES
    ('A','Иванов','Москва',111),
    ('A','Иванов','Москва',222),
    ('B','Петров','Иваново',333),
    ('C','Сидоров','Москва',444);

 * sqlite:////content//tempt.db
Done.
4 rows affected.


[]

In [80]:
%%sql
CREATE TABLE Accounts ([Date] date, Summa_USD money, Account_num int);
INSERT INTO Accounts ([Date], Summa_USD, Account_num) VALUES
('2012-01-01',15000,111),
('2012-02-01',10000,111),
('2012-02-01',5000,222),
('2012-03-01',30000,333),
('2012-04-01',20000,444);

 * sqlite:////content//tempt.db
Done.
5 rows affected.


[]

In [79]:
query = '''
WITH t AS (
SELECT
  s.*,
  c.*,
  sum(Summa_USD) OVER (PARTITION BY c.client_id ORDER BY [Date]) as cum_sum,
  row_number() OVER (partition BY s.Account_num ORDER BY [Date] DESC) as rn
FROM Cчета s
INNER JOIN Clients c
ON
  c.Account_num = s.Account_num
  AND c.region = 'Москва')

SELECT distinct(client_id)
FROM t
WHERE cum_sum >= 20000
AND rn = 1
'''
%sql $query

 * sqlite:////content//tempt.db
Done.


client_id
A
C


In [85]:
query = '''
SELECT
  c.client_id
FROM Clients c
LEFT JOIN Accounts a
ON a.Account_num = c.Account_num
WHERE c.Region = 'Москва'
GROUP BY c.client_id
HAVING sum(Summa_USD) >= 20000
'''
%sql $query

 * sqlite:////content//tempt.db
Done.


client_id
A
C


## SQL2
Необходимо посчитать кол-во клиентов, у которых баланс на 2023-07-07 больше 500
рублей.

* IN - приход средств на счет клиента,
* OUT - отток.

create table oper (client varchar, date date, type varchar, amount int);

insert into oper values
('a', '2023-06-02', 'in', 1985),

('a', '2023-06-03', 'in', 1577),

('a', '2023-06-04', 'out', 1597),

('b', '2023-06-05', 'in', 1468),

('c', '2023-07-06', 'in', 582),

('c', '2023-07-08', 'out', 550),

('d', '2023-06-09', 'in', 1556),

('d', '2023-06-10', 'out', 1480)

In [88]:
%%sql
DROP TABLE IF EXISTS oper;
CREATE TABLE oper (client varchar, [date] date, [type] varchar, amount int);
INSERT INTO oper (client, [date], [type], amount) VALUES
('a', '2023-06-02', 'in', 1985),
('a', '2023-06-03', 'in', 1577),
('a', '2023-06-04', 'out', 1597),
('b', '2023-06-05', 'in', 1468),
('c', '2023-07-06', 'in', 582),
('c', '2023-07-08', 'out', 550),
('d', '2023-06-09', 'in', 1556),
('d', '2023-06-10', 'out', 1480);

 * sqlite:////content//tempt.db
Done.
Done.
8 rows affected.


[]

In [129]:
sql = """
WITH t AS (
SELECT client,
SUM(CASE [type] WHEN 'in' THEN amount ELSE 0-amount END) summ
FROM oper
WHERE [date] <= '2023-07-07'
GROUP BY client
HAVING summ > 500)

SELECT COUNT(client) FROM t
"""
%sql $sql

 * sqlite:////content//tempt.db
Done.


COUNT(client)
3


In [8]:
df = pd.read_csv("https://github.com/aleks-haksly/Simulative/raw/refs/heads/main/Other/Analysis/%D0%90%D0%BB%D1%8C%D1%84%D0%B0%20%D0%91%D0%B0%D0%BD%D0%BA%20test/dataset.zip",
                 compression='zip', sep=';')

In [9]:
df.head()

,Unnamed: 0,CLIENT_ID,VALUE_DAY
0,0,3596869440,30.09.2021
1,1,181778433,30.09.2021
2,2,4271531990,30.09.2021
3,3,3949165697,30.09.2021
4,4,1926920317,30.09.2021


In [ ]:
exec_query = lambda query: sqldf(query, env=globals())

In [ ]:
exec_query("SELECT * FROM df where query like '%утро%'")

,platform,hour,query,count
0,desktop,0,доброе утро,1
